## **Install Necessary Packages**

In [ ]:
pip install moorcheh-sdk langchain_community pypdf pandas

In [ ]:
import os
import csv
import pandas as pd
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time
from sentence_transformers import SentenceTransformer  # Optional: Can be used for embedding
from google.colab import userdata
from moorcheh_sdk import MoorchehClient

## **Initalize the MoorchehClient**

In [ ]:
MOORCHEH_API_KEY = userdata.get("MOORCHEH_API_KEY")

namespace_name="new_namepsaces"
client = MoorchehClient(api_key=MOORCHEH_API_KEY)
client.create_namespace(namespace_name=namespace_name, type="text")

pdf_path = "CombinedDoc.pdf"
query_csv_path = "queries.csv" # Path to your CSV file with queries
output_csv_path = "answers.csv" # Where to save the results
top_k = 5

## **Prepare and Chunk the Documents**

In [ ]:
loader = PyPDFLoader(pdf_path)
pages = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=120)
chunks = splitter.split_documents(pages)

## **Upload the Document Chunks**

In [ ]:
documents = []
for i, chunk in enumerate(chunks):
    text = chunk.page_content.strip()
    documents.append({"id": f"chunk_{i}",
                      "text": text})

client.upload_documents(namespace_name=namespace_name, documents=documents)

{'status': 'success',
 'message': 'Successfully queued 211 documents for processing.',
 'queued_documents': 211,
 'document_ids': ['chunk_0',
  'chunk_1',
  'chunk_2',
  'chunk_3',
  'chunk_4',
  'chunk_5',
  'chunk_6',
  'chunk_7',
  'chunk_8',
  'chunk_9',
  'chunk_10',
  'chunk_11',
  'chunk_12',
  'chunk_13',
  'chunk_14',
  'chunk_15',
  'chunk_16',
  'chunk_17',
  'chunk_18',
  'chunk_19',
  'chunk_20',
  'chunk_21',
  'chunk_22',
  'chunk_23',
  'chunk_24',
  'chunk_25',
  'chunk_26',
  'chunk_27',
  'chunk_28',
  'chunk_29',
  'chunk_30',
  'chunk_31',
  'chunk_32',
  'chunk_33',
  'chunk_34',
  'chunk_35',
  'chunk_36',
  'chunk_37',
  'chunk_38',
  'chunk_39',
  'chunk_40',
  'chunk_41',
  'chunk_42',
  'chunk_43',
  'chunk_44',
  'chunk_45',
  'chunk_46',
  'chunk_47',
  'chunk_48',
  'chunk_49',
  'chunk_50',
  'chunk_51',
  'chunk_52',
  'chunk_53',
  'chunk_54',
  'chunk_55',
  'chunk_56',
  'chunk_57',
  'chunk_58',
  'chunk_59',
  'chunk_60',
  'chunk_61',
  'chunk_62',

## **Generate Answer**

In [ ]:
queries_df = pd.read_csv(query_csv_path) # Load your questions from a CSV file

with open(output_csv_path, "w", newline="") as f: # Open the results CSV file
    writer = csv.DictWriter(f, fieldnames=["passage_id", "query", "generated_answers"]) # Set up CSV columns
    writer.writeheader() # Write the column headers

    for idx, q in enumerate(queries_df["query"]): # Go through each question
        print(f"Processing: {q}") # Show which question is being processed
        try:
            response = client.get_generative_answer(query = q,
                                               ai_model = "anthropic.claude-3-7-sonnet-20250219-v1:0",
                                               namespace=namespace_name).get("answer", "")
            time.sleep(0.5)
            writer.writerow({ # Write the results to the CSV
                "passage_id": idx, # Unique ID for this answer
                "query": q, # The original question
                "generated_answers": response # The AI-generated answer
            })
        except Exception as e: # If something goes wrong
            print(f"Error for query '{q}':", e) # Print the error

Processing: What is Meta’s Net Income in 2025?
Processing: What is Netflix’s Profit in 2025?
Processing: What is Apple’s Expenses in 2025?
Processing: What is Alphabet’s Revenue in 2025?
Processing: What is Amazon’s operating expenses in 2025?
Processing: Which company had the highest revenue in the first quarter of 2025?
Processing: Which company has seen the least growth?
Processing: Which of the five companies showed the strongest overall financial performance in early 2025 and why?
Processing: Compare the quarter-on-quarter revenue growth between Meta and Alphabet.
Processing: Compare the quarter-on-quarter revenue growth between Netflix and Amazon.
